In [1]:
#Voice to Text - OpenAI’s Whisper Speech-to-Text APP

In [2]:
pip install openai pyaudio wave


     ---------------------------------------- 71.9/71.9 kB 1.9 MB/s eta 0:00:00
     -------------------------------------- 164.1/164.1 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 319.8/319.8 kB 4.9 MB/s eta 0:00:00
     ---------------------------------------- 77.1/77.1 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 61.0/61.0 kB 3.4 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1220 sha256=e910464d1daa9bf9b1fc41cee23d11284bb6fe2f1eff5df669030c10caa9b90c
  Stored in directory: c:\users\jonny\appdata\local\pip\cache\wheels\42\b2\dd\1b9c410804338d504c57108263448095a33995129e11ace1b1
Successfully built wave
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [21]:
import openai
import os
import pyaudio
import wave
import threading
import sys
import queue

# Setting my OpenAI API key
openai.api_key = ""

#Creating the record_audio function which saves it as a WAV file
def record_audio(filename, stop_event, audio_queue):
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    CHUNK = 1024

    audio = pyaudio.PyAudio()
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)

    print("Recording... Press return key to stop.")

    while not stop_event.is_set():
        data = stream.read(CHUNK)
        audio_queue.put(data)

    print("Finished recording")

    stream.stop_stream()
    stream.close()
    audio.terminate()

    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(audio.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(list(audio_queue.queue)))

#Creating the transcribe_audio function through Whisper API
def transcribe_audio(filename):
    with open(filename, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
        return transcript["text"]

#Defining the main function to transcribe the recorded audio
def main():
    audio_filename = "recorded_audio.wav"
    stop_event = threading.Event()
    audio_queue = queue.Queue()

    record_thread = threading.Thread(target=record_audio, args=(audio_filename, stop_event, audio_queue))
    record_thread.start()

    input("Press the return key to stop recording...\n")
    stop_event.set()
    record_thread.join()

    transcription = transcribe_audio(audio_filename)
    print("Transcription:", transcription)

#Running the app
if __name__ == "__main__":
    main()

Recording... Press return key to stop.
Finished recording


AuthenticationError: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.